In [ ]:
!pip install torch
!pip install scikit-learn
!pip install scikit-image
!pip install earthengine-api
!pip install rasterio
!pip install lmdb
!pip install pyshp
!pip install shapely
!pip install pytorch-lightning
!pip install pytorch-lightning-bolts
!pip install torchvision

In [ ]:
!git clone https://github.com/Akkarapon-chaiyana/ChangeDetection.git

In [ ]:
import ee
import os
import random
ee.Authenticate()
ee.Initialize(project = 'tony-1122')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Path to the folder you want to create
folder_path = '/content/drive/MyDrive/ChangeDetection'

# Create the folder if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created")
else:
    print("Folder already exists")

In [ ]:
# Path to the folder you want to create
folder_path = '/content/drive/MyDrive/ChangeDetection/Images'

# Create the folder if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created")
else:
    print("Folder already exists")

In [ ]:
# Base path for the folders
base_path = '/content/drive/MyDrive/ChangeDetection/Images'

# List of folder names to create
folders = ['before', 'after', 'label']

# Loop through the folder names and create each one
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder}' created")
    else:
        print(f"Folder '{folder}' already exists")

In [ ]:
# Path to the folder you want to create
folder_path = '/content/drive/MyDrive/ChangeDetection/Output'

# Create the folder if it does not exist
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print("Folder created")
else:
    print("Folder already exists")

In [ ]:
# Base path for the folders
base_path = '/content/drive/MyDrive/ChangeDetection/Output'

# List of folder names to create
folders = ['Inference', 'Prediction_TIFFs', 'Validations_Figures']

# Loop through the folder names and create each one
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder}' created")
    else:
        print(f"Folder '{folder}' already exists")

In [ ]:
%cd /content/ChangeDetection

In [ ]:
!python exports2AlertsGlad.py

In [ ]:
# Set paths
dataset_path = "/content/drive/MyDrive/ChangeDetection/Images/after"
train_file = "/content/drive/MyDrive/ChangeDetection/Images/train.txt"
test_file = "/content/drive/MyDrive/ChangeDetection/Images/test.txt"
all_file = "/content/drive/MyDrive/ChangeDetection/Images/all.txt"

# Parameters
train_split = 0.7

# Get the list of all folders or images in the dataset
all_files = os.listdir(dataset_path)

# Write all.txt
with open(all_file, 'w') as f:
    f.write(",".join(all_files))

random.shuffle(all_files)

# Split into train and test
train_size = int(len(all_files) * train_split)
train_files = all_files[:train_size]
test_files = all_files[train_size:]

# Write train.txt
with open(train_file, 'w') as f:
    f.write(",".join(train_files))

# Write test.txt
with open(test_file, 'w') as f:
    f.write(",".join(test_files))

In [ ]:
# ckp_path can download from https://github.com/zhu-xlab/SSL4EO-S12
!python main_oscd.py  \
--data_dir /content/drive/MyDrive/ChangeDetection/Images  \
--ckp_path /content/drive/MyDrive/SSL4EO/model/B13_rn18_moco_0099_ckpt.pth \
--result_dir /content/drive/MyDrive/ChangeDetection/Output \
--resnet_type 18 \
--n_channels 13 \
--n_epochs 10

In [ ]:
!python validate_oscd.py \
--data_dir /content/drive/MyDrive/ChangeDetection/Images \
--ckp_pretrain /content/drive/MyDrive/SSL4EO/model/B13_rn18_moco_0099_ckpt.pth \
--ckp_resume /content/drive/MyDrive/ChangeDetection/Output/ckps/random/*.ckpt \
--result_dir /content/drive/MyDrive/ChangeDetection/Output \
--n_channels 13 \
--n_epochs 10 \
--resnet_type 18

In [ ]:
from IPython.display import display
from PIL import Image

In [ ]:
def display_images_from_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".png") or filename.endswith(".tif"):  # checking file extension
            img_path = os.path.join(folder_path, filename)
            with Image.open(img_path) as img:
                display(img)
                #print(filename)  # Optional: to show the file name above the image

In [ ]:
folder_path = '/content/drive/MyDrive/ChangeDetection/Output/Validations_Figures'
display_images_from_folder(folder_path)

In [ ]:
## Remarks main_oscd_inference.py will read ckp_resume differently.
## We can edit it on Colab of git clone folder.
!python main_oscd_inference.py \
--data_dir /content/drive/MyDrive/ChangeDetection/Images \
--ckp_path /content/drive/MyDrive/SSL4EO/model/B13_rn18_moco_0099_ckpt.pth \
--result_dir /content/drive/MyDrive/ChangeDetection/Output \
--n_channels 13 \
--n_epochs 10 \
--resnet_type 18

In [ ]:
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import glob
import os

# Path to the folder containing .tif files
folder_path = '/content/drive/MyDrive/ChangeDetection/Output/Inference'

# List all .tif files in the folder
tif_files = glob.glob(os.path.join(folder_path, '*.tif'))

In [ ]:
# Number of columns for the subplot
columns = 4
# Calculate number of rows needed
rows = len(tif_files) // columns + (1 if len(tif_files) % columns else 0)

# Create a figure with subplots
plt.figure(figsize=(15, rows * 5))

for i, file in enumerate(tif_files):
    with rasterio.open(file) as src:
        # Read the image data
        image = src.read()
        # Create subplot
        ax = plt.subplot(rows, columns, i + 1)
        # Display the image
        show(image, ax=ax, transform=src.transform)
        # Set title as filename
        ax.set_title(os.path.basename(file))
        # Remove x and y ticks
        ax.set_xticks([])
        ax.set_yticks([])

plt.tight_layout()
plt.show()
